###*Union de tablas para Funciones Endpoint*

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')

###Importamos Archivos

In [2]:
archivo="steam_games.parquet"
steam_games= pd.read_parquet(archivo)
steam_games.head()

,app_name,price,item_id,developer,release_year,genres
0,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018,Action
1,Ironbound,0.00,643980,Secret Level SRL,2018,Free to Play
2,Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017,Casual
3,弹炸人2222,0.99,767400,彼岸领域,2017,Action
4,Battle Royale Trainer,3.99,772540,Trickjump Games Ltd,2018,Action


In [3]:
archivo="australian_user_reviews.parquet"
user_reviews= pd.read_parquet(archivo)
user_reviews.head()

,user_id,item_id,recommend,posted_year,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,1
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,2


In [4]:
archivo="australian_users_items.parquet"
user_items= pd.read_parquet(archivo)
user_items.head()

,item_id,item_name,steam_id,items_count,user_id,playtime_hours
0,10,Counter-Strike,76561197970982479,277,76561197970982479,0.10
1,30,Day of Defeat,76561197970982479,277,76561197970982479,0.12
2,300,Day of Defeat: Source,76561197970982479,277,76561197970982479,78.88
3,240,Counter-Strike: Source,76561197970982479,277,76561197970982479,30.88
4,3830,Psychonauts,76561197970982479,277,76561197970982479,5.55


###1- def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

In [5]:
# price, año, item_id
df_merge = pd.merge(steam_games, user_reviews, left_on='item_id', right_on='item_id', how='inner')  # Aquí puedes elegir el método de combinación (inner, outer, left, right)
df_merge

,app_name,price,item_id,developer,release_year,genres,user_id,recommend,posted_year,sentiment_analysis
0,Carmageddon Max Pack,9.99,282010,Stainless Games Ltd,1997,Action,BlazingTheAmazing,True,2013,1
1,Half-Life,9.99,70,Valve,1998,Action,exiaez,True,2015,1
2,Half-Life,9.99,70,Valve,1998,Action,mrpfresh,True,2011,0
3,Half-Life,9.99,70,Valve,1998,Action,armouredmarshmallow,True,2014,1
4,Half-Life,9.99,70,Valve,1998,Action,Bluegills,True,2013,2
...,...,...,...,...,...,...,...,...,...,...
50325,Counter-Strike: Condition Zero,9.99,80,Valve,2004,Action,76561198023226082,False,2014,1
50326,Counter-Strike: Condition Zero,9.99,80,Valve,2004,Action,76561198030797558,True,2013,2
50327,Counter-Strike: Condition Zero,9.99,80,Valve,2004,Action,Deadly909,True,2015,1
50328,Counter-Strike: Condition Zero,9.99,80,Valve,2004,Action,76561198099442511,True,2014,1


In [6]:
df_desarrollador = df_merge.drop(columns=["app_name", "posted_year", "genres", "recommend", "user_id", "sentiment_analysis"])
df_desarrollador

,price,item_id,developer,release_year
0,9.99,282010,Stainless Games Ltd,1997
1,9.99,70,Valve,1998
2,9.99,70,Valve,1998
3,9.99,70,Valve,1998
4,9.99,70,Valve,1998
...,...,...,...,...
50325,9.99,80,Valve,2004
50326,9.99,80,Valve,2004
50327,9.99,80,Valve,2004
50328,9.99,80,Valve,2004


In [7]:
tabla = pa.Table.from_pandas(df_desarrollador) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"1_desarrollador.parquet") # guardamos la tabla en formato parquet

In [8]:
df_desarrollador.to_csv("1_desarrollador.csv", index=False, encoding="utf-8") 

###2- def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [9]:
df_User_id = df_merge.drop(columns=["app_name", "item_id",	"developer", "genres", "release_year", "sentiment_analysis"])
df_User_id

,price,user_id,recommend,posted_year
0,9.99,BlazingTheAmazing,True,2013
1,9.99,exiaez,True,2015
2,9.99,mrpfresh,True,2011
3,9.99,armouredmarshmallow,True,2014
4,9.99,Bluegills,True,2013
...,...,...,...,...
50325,9.99,76561198023226082,False,2014
50326,9.99,76561198030797558,True,2013
50327,9.99,Deadly909,True,2015
50328,9.99,76561198099442511,True,2014


In [10]:
tabla = pa.Table.from_pandas(df_User_id) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"2User_id.parquet") # guardamos la tabla en formato parquet

In [11]:
df_User_id.to_csv("2User_id.csv", index=False, encoding="utf-8") 

###3- def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [12]:
# genres, release_año, playtime_hours
df_merge2 = pd.merge(df_merge, user_items, on =['user_id','item_id'], how='inner')  # Aquí puedes elegir el método de combinación (inner, outer, left, right)
df_merge2

,app_name,price,item_id,developer,release_year,genres,user_id,recommend,posted_year,sentiment_analysis,item_name,steam_id,items_count,playtime_hours
0,Half-Life,9.99,70,Valve,1998,Action,mrpfresh,True,2011,0,Half-Life,76561197974266367,791,2.32
1,Half-Life,9.99,70,Valve,1998,Action,armouredmarshmallow,True,2014,1,Half-Life,76561198104386187,119,13.13
2,Half-Life,9.99,70,Valve,1998,Action,Bluegills,True,2013,2,Half-Life,76561197971857001,1280,1.07
3,Half-Life,9.99,70,Valve,1998,Action,76561198071955492,True,2013,2,Half-Life,76561198071955492,123,53.92
4,Half-Life,9.99,70,Valve,1998,Action,digfernandes,True,2014,1,Half-Life,76561198047268909,567,21.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12930,Counter-Strike,9.99,10,Valve,2000,Action,timtam012,True,2014,1,Counter-Strike,76561198055650636,144,24.28
12931,Counter-Strike,9.99,10,Valve,2000,Action,Zanscape,True,2015,2,Counter-Strike,76561198066331075,258,0.02
12932,Unreal Tournament 2004: Editor's Choice Edition,14.99,13230,"Epic Games, Inc.",2004,Action,76561197994404698,True,2012,2,Unreal Tournament 2004,76561197994404698,77,12.90
12933,Unreal Tournament 2004: Editor's Choice Edition,14.99,13230,"Epic Games, Inc.",2004,Action,76561198011218089,True,2014,1,Unreal Tournament 2004,76561198011218089,389,43.10


In [13]:
df_genero = df_merge2.drop(columns=["app_name", "price","posted_year","item_id","steam_id","items_count", "developer", "item_id", "recommend", "sentiment_analysis", "item_name"])
df_genero

,release_year,genres,user_id,playtime_hours
0,1998,Action,mrpfresh,2.32
1,1998,Action,armouredmarshmallow,13.13
2,1998,Action,Bluegills,1.07
3,1998,Action,76561198071955492,53.92
4,1998,Action,digfernandes,21.68
...,...,...,...,...
12930,2000,Action,timtam012,24.28
12931,2000,Action,Zanscape,0.02
12932,2004,Action,76561197994404698,12.90
12933,2004,Action,76561198011218089,43.10


In [14]:
tabla = pa.Table.from_pandas(df_genero) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"3genero.parquet") # guardamos la tabla en formato parquet

In [15]:
df_genero.to_csv("3genero.csv", index=False, encoding="utf-8") 

###4- def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [16]:

# developer, recommend, user_id, año, "sentiment_analysis"
df_año = df_merge.drop(columns=["app_name", "price", "item_id","posted_year","genres", "item_id"])
df_año                       

,developer,release_year,user_id,recommend,sentiment_analysis
0,Stainless Games Ltd,1997,BlazingTheAmazing,True,1
1,Valve,1998,exiaez,True,1
2,Valve,1998,mrpfresh,True,0
3,Valve,1998,armouredmarshmallow,True,1
4,Valve,1998,Bluegills,True,2
...,...,...,...,...,...
50325,Valve,2004,76561198023226082,False,1
50326,Valve,2004,76561198030797558,True,2
50327,Valve,2004,Deadly909,True,1
50328,Valve,2004,76561198099442511,True,1


In [17]:
tabla = pa.Table.from_pandas(df_año) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"4año.parquet") # guardamos la tabla en formato parquet

In [18]:
df_año.to_csv("4año.csv", index=False, encoding="utf-8") 

###5-def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [19]:
# developer, sentiment_analysis
df_desarrolladora = df_merge.drop(columns=["app_name", "price", "item_id", "genres", "item_id","posted_year","recommend", "user_id", "release_year"])
df_desarrolladora                           

,developer,sentiment_analysis
0,Stainless Games Ltd,1
1,Valve,1
2,Valve,0
3,Valve,1
4,Valve,2
...,...,...
50325,Valve,1
50326,Valve,2
50327,Valve,1
50328,Valve,1


In [20]:
tabla = pa.Table.from_pandas(df_desarrolladora) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"5desarrolladora.parquet") # guardamos la tabla en formato parquet

In [21]:
df_desarrolladora.to_csv("5desarrolladora.csv", index=False, encoding="utf-8")